In [9]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

***My Working example for types of run***

In [11]:
from langsmith import traceable
from openai import OpenAI
from typing import List, Optional
import json

# Initialize the OpenAI client
openai_client = OpenAI()

@traceable()
def summarize_text(text: str) -> str:
    """
    Summarizes a given text string.
    Example: "LangSmith helps in debugging and tracing applications" ->
             "LangSmith simplifies debugging and tracing."
    """
    # Simple summarization logic for demo purposes
    words = text.split()
    summary = " ".join(words[:10]) + ("..." if len(words) > 10 else "")
    return summary

@traceable()
def call_openai_summary(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
    """Wrapper for the OpenAI API call."""
    return openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools
    )

@traceable(run_type="chain")
def summarize_paragraph(inputs: List[dict], tools: List[dict]):
    """
    Main chain where the LLM identifies a summarization task,
    calls the summarization tool, and returns the final output.
    """
    response = call_openai_summary(inputs, tools)

    tool_call = response.choices[0].message.tool_calls[0]
    tool_call_args = json.loads(tool_call.function.arguments)

    text_to_summarize = tool_call_args["text"]

    summary_result = summarize_text(text_to_summarize)

    tool_response_message = {
        "role": "tool",
        "content": json.dumps({
            "text": text_to_summarize,
            "summary": summary_result,
        }),
        "tool_call_id": tool_call.id
    }

    inputs.append(response.choices[0].message)
    inputs.append(tool_response_message)

    output = call_openai_summary(inputs, None)
    return output


# Tool definition for LangSmith trace
summary_tools = [
    {
        "type": "function",
        "function": {
            "name": "summarize_text",
            "description": "Generate a short summary of a long paragraph.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The paragraph or text to summarize."
                    }
                },
                "required": ["text"]
            }
        }
    }
]

# Input messages for the LLM
summary_inputs = [
    {"role": "system", "content": "You are a helpful assistant that summarizes text using the 'summarize_text' tool."},
    {"role": "user", "content": "Please summarize the following paragraph:\n\nLangSmith makes it easier to monitor, debug, and optimize LLM-powered applications by providing detailed traces and metadata for every function call."}
]

# The corrected part: You must call the main function for the trace to be generated and logged.
if __name__ == "__main__":
    final_output = summarize_paragraph(summary_inputs, summary_tools)
    print(final_output.choices[0].message.content)

LangSmith simplifies the monitoring, debugging, and optimization of LLM-powered applications by offering detailed traces and metadata for each function call.


This is the link of the run i have shared - 
https://smith.langchain.com/public/40f018dd-cfd3-400a-ae8a-1c2580a44d2f/r